In [ ]:
# 초기 세팅
import pandas as pd

master = pd.read_csv("../dataset/master_4.csv", encoding='utf-8')

In [2]:
master = master.drop(["intime", "outtime", "ed_los", "outcome_icu_transfer_12h"], axis='columns')

master_X = master.loc[:, master.columns!='outcome_critical']
master_Y = master['outcome_critical']

master_Y.replace([False, True],[0,1], inplace=True)

In [3]:
# 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
master_X_std = scaler.fit_transform(master_X)

In [4]:
import numpy as np

cov_mat = np.cov(master_X_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

In [5]:
# tuple list (eigen value, eigen vector)
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:, i])
               for i in range(len(eigen_vals))]

eigen_pairs.sort(key = lambda k: k[0], reverse = True)

In [6]:
from sklearn.decomposition import PCA

# 전체 주성분에 대한 분산의 크기
pca = PCA(n_components = None)
master_X_std = pca.fit_transform(master_X_std)
pca.explained_variance_ratio_ # 내림차순으로 나타남

array([1.18197341e-01, 4.91122043e-02, 3.34946376e-02, 2.65472028e-02,
       2.55268656e-02, 2.51630375e-02, 2.26207331e-02, 2.22319482e-02,
       2.13205249e-02, 2.06063984e-02, 1.97520465e-02, 1.93487015e-02,
       1.79192386e-02, 1.70137056e-02, 1.68712773e-02, 1.63273354e-02,
       1.58730404e-02, 1.51673740e-02, 1.31861968e-02, 1.27298967e-02,
       1.24237273e-02, 1.21152583e-02, 1.16749227e-02, 1.15271107e-02,
       1.12501250e-02, 1.10175851e-02, 1.05892344e-02, 1.04238091e-02,
       1.03314345e-02, 1.02303098e-02, 1.01953520e-02, 1.01872265e-02,
       1.01582552e-02, 1.01496367e-02, 9.89458282e-03, 9.78540653e-03,
       9.71121082e-03, 9.49911766e-03, 9.26476918e-03, 9.15666766e-03,
       9.04363501e-03, 8.90817124e-03, 8.84837258e-03, 8.66493422e-03,
       8.48932065e-03, 8.35472647e-03, 8.27352319e-03, 8.18701700e-03,
       7.99108838e-03, 7.90570299e-03, 7.74879023e-03, 7.63074170e-03,
       7.53149525e-03, 7.35537372e-03, 7.21803332e-03, 7.12551474e-03,
      

In [8]:
# for i in range(2, 50, 2):
#     pca = PCA(n_components=i) # 주성분을 몇개로 할지 결정
#     master_X_pca = pca.fit_transform(master_X_std)
#     PCA_X = pd.DataFrame(data=master_X_pca)
#     # 주성분으로 이루어진 데이터 프레임 구성

#     print("----------" + str(i) + "----------")
#     print("성분 별 분산 설명 비율 :", pca.explained_variance_ratio_)
#     print("전체 분산 설명 비율 :", sum(pca.explained_variance_ratio_))
#     print()

In [7]:
print("분산 설명력 0.6 :", 27)
print("분산 설명력 0.7 :", 37)

분산 설명력 0.6 : 27
분산 설명력 0.7 : 37


In [9]:
pca = PCA(n_components=27)
master_X_std_27 = pca.fit_transform(master_X_std)
PCA_X_27 = pd.DataFrame(data=master_X_std_27)

In [8]:
pca = PCA(n_components=37)
master_X_std_37 = pca.fit_transform(master_X_std)
PCA_X_37 = pd.DataFrame(data=master_X_std_37)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(master_X_std, master_Y, test_size=0.33, random_state=321)
X_train_27, X_test_27, Y_train_27, Y_test_27 = train_test_split(PCA_X_27, master_Y, test_size=0.33, random_state=321)
X_train_37, X_test_37, Y_train_37, Y_test_37 = train_test_split(PCA_X_37, master_Y, test_size=0.33, random_state=321)

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def setting(y_pred):
    for i in range(len(y_pred)):
        if y_pred[i]>=.5:       # setting threshold to .5
           y_pred[i]=1
        else:  
           y_pred[i]=0

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f},\nF1: {3: .4f}, AUC: {4: .4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

print("Default\n")
lr.fit(X_train, Y_train)
y_pred = lr.predict(X_test)
setting(y_pred)
get_clf_eval(Y_test, y_pred)

print("\n------------------")
print("27\n")

lr.fit(X_train_27, Y_train_27)
y_pred = lr.predict(X_test_27)
setting(y_pred)
get_clf_eval(Y_test, y_pred)

print("\n------------------")
print("37\n")

lr.fit(X_train_37, Y_train_37)
y_pred = lr.predict(X_test_37)
setting(y_pred)
get_clf_eval(Y_test, y_pred)

Default



C:\Users\Seok\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


오차 행렬
[[137246   1009]
 [  6236   3670]]
정확도:  0.9511, 정밀도:  0.7844, 재현율:  0.3705,
F1:  0.5033, AUC:  0.6816

------------------
27

오차 행렬
[[137297    958]
 [  7669   2237]]
정확도:  0.9418, 정밀도:  0.7002, 재현율:  0.2258,
F1:  0.3415, AUC:  0.6094

------------------
37

오차 행렬
[[137447    808]
 [  7391   2515]]
정확도:  0.9447, 정밀도:  0.7568, 재현율:  0.2539,
F1:  0.3802, AUC:  0.6240
